In [1]:
# NOTE: This uses a modified ethereum-etl here: https://github.com/MSilb7/optimism-etl

import sys
import json
from web3 import Web3
from web3.middleware import geth_poa_middleware
sys.path.append("../helper_functions")
import ethereumetl_utils as eetl
import web3py_utils as wpy
import pandas_utils as pu
import evm_utils as evm
sys.path.pop()

import pandas as pd
from dotenv import load_dotenv
import requests
import datetime
load_dotenv()
import os

configs = [
        #  ['ethereum',os.environ.get('ETHEREUM_NODE'),'0x2bd9c2576fd499909a96965318a176c5d2d0edd2c7be2f481305c55dc5327bad']
          ['optimism-goerli-ecotone',os.environ.get('OP_GOERLI'),'0x6f6dfc6d1e6948eb804c0ee932caa33fa330ab905993a05bba5c28f4f958d3e3']
          # ,['optimism-legacy',os.environ.get('OP_NODE'),'0xf659ca817904644883c4180c68ca470ffda9d1bb6f3003e5be7feb8580d1f0c0']
]

In [2]:
for c in configs:
        chain = c[0]
        print('-------' + chain + '-------')
        uri = c[1]
        api = os.environ.get('ALCHEMY_OP_API_KEY')
        # Connect to a local Ethereum node
        web3 = Web3(Web3.HTTPProvider(uri))

        # Specify the transaction hash
        transaction_hash = c[2]

        # Get the transaction receipt
        receipt = web3.eth.getTransactionReceipt(transaction_hash)
        print("Transaction Receipt:")
        print(receipt)

        # Get the transaction result
        transaction = web3.eth.getTransaction(transaction_hash)
        print("\nTransaction Result:")
        print(transaction)

        # Get the block number from the transaction receipt
        block_number = receipt['blockNumber']
        # print("\nBlock Number:")
        # print(block_number)
        
        if chain == 'optimism-legacy': # Legacy Optimism returned null for block data
                web3.middleware_onion.inject(geth_poa_middleware, layer=0)
                block = web3.eth.getBlock(block_number, full_transactions=False)
        else:
                block = web3.eth.getBlock(block_number, full_transactions=False)

        print("\nBlock Data:")
        print(block)

        gas_price = transaction['gasPrice']
        try:
                base_fee = block['baseFeePerGas']
        except:
                base_fee = 0.001

        tip = gas_price - base_fee

        print('\n')
        # Print

        # print('gas price: ' + str(gas_price/1e9))
        # print('base fee: ' + str(base_fee/1e9))
        # print('tip: ' + str(tip/1e9))


-------optimism-goerli-ecotone-------
Transaction Receipt:
AttributeDict({'blockHash': HexBytes('0x5050c1402f9f3fa4960bf64b80d0e7f119069d1127b655e74a357020ff66435a'), 'blockNumber': 21157590, 'contractAddress': None, 'cumulativeGasUsed': 165376, 'effectiveGasPrice': 1000252, 'from': '0x4e92B2A0376E14940417DE5c0Ca55AE613ca0351', 'gasUsed': 121537, 'l1Fee': '0x53ada8fe144', 'l1GasPrice': '0x7', 'l1GasUsed': '0x1794', 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'status': 1, 'to': '0x7d05d5